In [26]:
%matplotlib inline 

In [27]:
import math
from typing import Tuple
import torch
from torch import nn,Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

In [28]:
import logging
import os
import torchvision
from torch.utils.data import DataLoader

_logger = logging.getLogger("app")
class DatasetVerify(object):

    def __init__(self, task_id, current_node):
        #初始化方法的列表参数不可以修改
        self.task_id = task_id
        self.current_node = current_node

    def run(self) -> bool:
        #用于检查是否能够访问到数据以及能够正确的将数据转化为特征
        """数据集的校验具体逻辑."""
        _logger.info(f"start dataset verification for task {self.task_id} on {self.current_node}")
       # root_dir = '/data/MNIST/'
        return self._touch_data(root_dir) and self._load_data(root_dir)
#验证访问和转换数据的模板
    def _touch_data(self, root_dir: str) -> bool:
        '''file_list = [
            't10k-images-idx3-ubyte',
            't10k-images-idx3-ubyte.gz',
            't10k-labels-idx1-ubyte',
            't10k-labels-idx1-ubyte.gz',
            'train-images-idx3-ubyte',
            'train-images-idx3-ubyte.gz',
            'train-labels-idx1-ubyte',
            'train-labels-idx1-ubyte.gz'
        ]
        full_paths = [os.path.join(root_dir, _file) for _file in file_list]
        return all(os.path.exists(_file) and os.path.isfile(_file)
                    for _file in full_paths)'''
        return True

    def _load_data(self, root_dir: str) -> bool:
        """data_loader = DataLoader(
            torchvision.datasets.MNIST(
                root_dir,
                train=True,
                download=False,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            )
        )"""
       # return data_loader is not None and len(data_loader) > 0
        return True

In [43]:
#定义模型
class TransformerModel(nn.Module):
    def __init__(self,ntoken:int,d_model:int, nhead: int, d_hid: int,
                nlayers: int,dropout: float = 0.5 ):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

#生成一个上三角矩阵
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

def make_model(ntoken:int,d_model:int, nhead: int, d_hid: int,
                nlayers: int,dropout: float = 0.5):
    return TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)



In [45]:
import os
from typing import Any, Dict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader



def state_dict(self) -> Dict[str, torch.Tensor]:
    return self.model.state_dict()

def load_state_dict(self, state_dict: Dict[str, torch.Tensor]):
    self.model.load_state_dict(state_dict)

In [46]:
#注入一些关于序列中的标记的相对或绝对位置的信息
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [47]:
import re
import torch 
from torch.utils.data import Dataset, DataLoader
import numpy as np
class MyDataset(Dataset):
    def __init__(self,data_set):
        super(MyDataset, self).__init__()
        self.data_set = data_set
        self.length = len(data_set)
    def __len__(self):
        return self.length
    
    def __getitem__(self,index):
        pass
        

In [48]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

def batchify(data: Tensor, bsz: int) -> Tensor: #给定一个顺序数据的一维向量，batchify()将数据排列成batch_size
    # 列。如果数据没有均匀地分成 batch_size列，那么数据将被修剪以适应
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """ 
    
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    
    return data.to(device)

#原始文本转换为平面张量
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    data = [torch.tensor(vocab(tokenizer(item)),dtype=torch.long)for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel()>0,data)))



train_iter = WikiText2(split='train') #获取训练集进行词典的构建
tokenizer = get_tokenizer('basic_english') #获取分词器
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>']) #创建词典，特殊词被定义为unk
vocab.set_default_index(vocab['<unk>']) #设置索引



def make_train_dataloader(self) -> DataLoader:
    return DataLoader(
        train_iter,
        train=True,
        batch_size=self.batch_size,
        shuffle=True
    )

def make_test_dataloader(self)-> DataLoader:
    return DataLoader(
        test_iter,
        train=True,
        batch_size=self.batch_size,
        shuffle=True
    )
def make_val_dataloader(self)-> DataLoader:
    return DataLoader(
        test_iter,
        train=True,
        batch_size=self.batch_size,
        shuffle=True
    )
"""def make_test_dataloader(bsz: int, test_iter: dataset.IterableDataset):
    test_data=data_process(test_iter)
    return batchify(test_data,bsz)
    """

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iter,val_iter,test_iter = WikiText2(root="D:\pywork\data\wikitext-2-v1")#在这里加root
batch_size = 20
eval_batch_size=10
#实在不明白如何使用dataloader，怎么使用本地wikitext2数据集，下载下来为token文件，打不开不知道具体格式，不知道该怎么处理
#很一头雾水
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)
#train_data=make_train_dataloader(batch_size,train_iter)
#val_data=make_test_dataloader(eval_batch_size,val_iter)
#test_data=make_test_dataloader(eval_batch_size,test_iter)


In [49]:
'''#from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

def batchify(data: Tensor, bsz: int) -> Tensor: #给定一个顺序数据的一维向量，batchify()将数据排列成batch_size
    # 列。如果数据没有均匀地分成 batch_size列，那么数据将被修剪以适应
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """ 
    
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    
    return data.to(device)

train_iter = WikiText2(split='train') #获取训练集进行词典的构建
tokenizer = get_tokenizer('basic_english') #获取分词器
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>']) #创建词典，特殊词被定义为unk
vocab.set_default_index(vocab['<unk>']) #设置索引

#原始文本转换为平面张量
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    data = [torch.tensor(vocab(tokenizer(item)),dtype=torch.long)for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel()>0,data)))

def make_train_dataloader(bsz: int, train_iter: dataset.IterableDataset):

    train_data=data_process(train_iter)
    return batchify(train_data,bsz)

def make_test_dataloader(bsz: int, test_iter: dataset.IterableDataset):
    test_data=data_process(test_iter)
    return batchify(test_data,bsz)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iter,val_iter,test_iter = WikiText2()
batch_size = 20
eval_batch_size=10
train_data=make_train_dataloader(batch_size,train_iter)
val_data=make_test_dataloader(eval_batch_size,val_iter)
test_data=make_test_dataloader(eval_batch_size,test_iter)

print(type(train_iter))
'''

'#from torchtext.datasets import WikiText2\nfrom torchtext.data.utils import get_tokenizer\nfrom torchtext.vocab import build_vocab_from_iterator\n\ndef batchify(data: Tensor, bsz: int) -> Tensor: #给定一个顺序数据的一维向量，batchify()将数据排列成batch_size\n    # 列。如果数据没有均匀地分成 batch_size列，那么数据将被修剪以适应\n    """Divides the data into bsz separate sequences, removing extra elements\n    that wouldn\'t cleanly fit.\n\n    Args:\n        data: Tensor, shape [N]\n        bsz: int, batch size\n\n    Returns:\n        Tensor of shape [N // bsz, bsz]\n    """ \n    \n    seq_len = data.size(0) // bsz\n    data = data[:seq_len * bsz]\n    data = data.view(bsz, seq_len).t().contiguous()\n    \n    return data.to(device)\n\ntrain_iter = WikiText2(split=\'train\') #获取训练集进行词典的构建\ntokenizer = get_tokenizer(\'basic_english\') #获取分词器\nvocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=[\'<unk>\']) #创建词典，特殊词被定义为unk\nvocab.set_default_index(vocab[\'<unk>\']) #设置索引\n\n#原始文本转换为平面张量\ndef data_process(raw_te

In [50]:
bptt = 35
#为转换器模型生成一对输入-目标序列，将源数据细分为长度的bptt
def get_batch(source: Tensor, i: int) ->Tuple[Tensor,Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int
        
    Returns:
        tuple(data,target),where data has shapem [seq_len,batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt,len(source)- 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data,target


In [51]:
#模型超参数
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = make_model(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [52]:
#训练模型，使用带有SGD优化器的CrossEntropyLoss。学习率最初设为5.0，遵循StepLR，
#训练期间使用nn.util.clip_grad_norm_ 来防止梯度爆炸
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # only on last batch
            src_mask = src_mask[:seq_len, :seq_len]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def test(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [ ]:
#循环遍历，如果验证是我们看到的最好的就保存模型，在每个epoch后调整学习率
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = test(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 453.86 | loss  8.29 | ppl  3974.02
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 543.18 | loss  6.92 | ppl  1008.79
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 515.38 | loss  6.45 | ppl   632.90
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 528.39 | loss  6.32 | ppl   553.28
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 613.36 | loss  6.19 | ppl   489.80
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 557.67 | loss  6.16 | ppl   471.94
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 581.54 | loss  6.12 | ppl   452.95


In [ ]:
test_loss = test(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

In [ ]:
make_model：返回训练使用的模型对象
在此过程中了解 PyTorch 中搭建多层网络结构的方式。
make_train_dataloader：返回训练数据的 DataLoader 对象
make_test_dataloader：返回测试数据的 DataLoader 对象
在此过程中了解 Dataset、DataLoader，以及二者之间的联系与区别。
train：模型训练方法
test：模型测试方法
